<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/hosting%20server%20in%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install simpletransformers
!pip install torch
!pip install pythainlp
!pip install python-crfsuite

In [3]:
!rm -rf '/content/ngrok-stable-linux-amd64.tgz'
!rm -rf '/content/ngrok'

!pip install flask --quiet
!pip install flask-ngrok --quiet

# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

# extract the downloaded file using the following command 
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

# paste your AuthToken here and execute this command
!./ngrok authtoken 23H0IY10fqeKMIW7kG05JhKZMae_3Zabr2iqkU9AUcZ7CrRTP

--2023-05-22 05:59:52--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  19.1MB/s    in 0.7s    

2023-05-22 05:59:53 (19.1 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
import logging
import torch
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 100
model_args.length_penalty = 1
model_args.num_beams = 10

device = torch.cuda.is_available()
path = '/content/gdrive/MyDrive/model/mt5-4bs-4e-5lr'

model = T5Model("mt5", path, args=model_args, use_cuda=device)

In [5]:
import re
import pycrfsuite
from pythainlp import sent_tokenize

def split_sentence_TH(TH_predict):
  TH_predict = sent_tokenize(TH_predict)
  for i in range(len(TH_predict)):
    # remove space in back
    TH_predict[i] = TH_predict[i].rstrip()

    # debug for engine fail example ['750ml', '1500ml']
    # move to append previous sentences
    # and replace ''
    if re.search('[0-9]', TH_predict[i][0]):
      j = i
      while True:
        if TH_predict[j-1] != '':
          TH_predict[j-1] += ' ' + TH_predict[i]
          break
      j -= 1
      TH_predict[i] = ''

  # remove '' in list
  while '' in TH_predict:
    TH_predict.remove('')
  print('[split sentence completed]')
  print(TH_predict)
  return TH_predict

def predict(text):
  predictions = model.predict(text)
  print('[split sentence completed]')
  print(predictions)
  return predictions

In [6]:
# import Flask from flask module
from flask import Flask, request, jsonify

# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok
  
app = Flask(__name__) #app name
run_with_ngrok(app)
  
@app.route("/", methods=["GET", "POST"])
def index():
  if request.method == "POST":
    json_data = request.get_json()
    if not json_data:
      return jsonify({"error": "no data"})
    try:
      if json_data["task"] == "th-en":
        prediction = split_sentence_TH(json_data["text"])
        prediction = predict(prediction)
        data_predicted = {"prediction": prediction}
        return jsonify(data_predicted)
        print('[returned data]')
    except Exception as e:
      return jsonify({"error": str(e)})
  return "OK"
  
if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packag